In [18]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')

In [19]:
df = pd.read_csv('Cleaned Money Management.csv')
df.head()

,Date,Details,NTD_Amount,Country,Currency,Foreign_Amount,Month,Year,Category
0,2023-01-01,KFC-台中中清,165,TW,TWD,0.0,1,2023,Junk Food
1,2023-01-01,覓靜拾光餐廳,1440,TW,TWD,0.0,1,2023,unassigned
2,2023-01-04,國外交易手續費,36,INT,NTD,0.0,1,2023,Digital Games
3,2023-01-04,GOOGLEANIPLEX,2373,JP,JPY,10000.0,1,2023,Digital Games
4,2023-01-07,必勝客,1544,TW,TWD,0.0,1,2023,Junk Food


In [20]:
# latest_month = df['Month'].max()
# latest_year = df['Year'].max()
# last_month_expenses = df[(df['Month'] == latest_month) & (df['Year'] == latest_year)]
# last_month_expenses

In [21]:
# last_month_expenses = last_month_expenses.groupby('Category')['NTD_Amount'].sum().reset_index()

# last_month_expenses['NTD_Amount']=last_month_expenses['NTD_Amount'].astype('str')
# last_month_expenses['NTD_Amount']=last_month_expenses['NTD_Amount'].str.replace('-','')
# last_month_expenses['NTD_Amount']=last_month_expenses['NTD_Amount'].astype('float')

# last_month_expenses = last_month_expenses[last_month_expenses["Category"].str.contains("unassigned") == False]
# last_month_expenses = last_month_expenses.sort_values(by='NTD_Amount', ascending=False)
# last_month_expenses['NTD_Amount'] = last_month_expenses['NTD_Amount'].round().astype(int)

# last_month_expenses

In [22]:
# feb_2024_expenses = df[(df['Month'] == 2) & (df['Year'] == 2024)]
# print("Entries for February 2024:")
# print(feb_2024_expenses)

In [23]:
# print("Latest Month:", latest_month)
# print("Latest Year:", latest_year)

In [24]:
latest_month = df['Month'].max()
latest_year = df[df['Month'] == latest_month]['Year'].max()
last_month_expenses = df[(df['Month'] == latest_month) & (df['Year'] == latest_year)]
if last_month_expenses.empty:
    print("No expenses found for February 2024")
else:
    # Group expenses by category and sum the NTD_Amount
    last_month_expenses = last_month_expenses.groupby('Category')['NTD_Amount'].sum().reset_index()

    # Filter out rows with 'unassigned' category
    last_month_expenses = last_month_expenses[last_month_expenses["Category"].str.contains("unassigned") == False]

    # Sort expenses by NTD_Amount in descending order
    last_month_expenses = last_month_expenses.sort_values(by='NTD_Amount', ascending=False)

    # Round NTD_Amount and convert to integer
    last_month_expenses['NTD_Amount'] = last_month_expenses['NTD_Amount'].round().astype(int)

    print("Expenses for February 2024:")
    print(last_month_expenses)

Expenses for February 2024:
        Category  NTD_Amount
5        Travels       20884
2      Groceries        2595
0       Delivery        1619
3      Junk Food         855
4         Sports         700
1  Digital Games         171


In [25]:
last_month_expenses_tot = last_month_expenses['NTD_Amount'].sum()
last_month_expenses_tot

26824

In [26]:
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

In [27]:
def calculate_difference(event):
    income = float(income_widget.value)
    recurring_expenses = float(recurring_expenses_widget.value)
    monthly_expenses = float(monthly_expenses_widget.value)
    difference = income - recurring_expenses - monthly_expenses
    difference_widget.value = str(difference)

income_widget = pn.widgets.TextInput(name="Income", value="0")
recurring_expenses_widget = pn.widgets.TextInput(name="Recurring Expenses", value="0")
monthly_expenses_widget = pn.widgets.TextInput(name="Non-Recurring Expenses", value=str(last_month_expenses_tot))
difference_widget = pn.widgets.TextInput(name="Last Month's Savings", value="0")

income_widget.param.watch(calculate_difference, "value")
recurring_expenses_widget.param.watch(calculate_difference, "value")
monthly_expenses_widget.param.watch(calculate_difference, "value")

# pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget).show()

Watcher(inst=TextInput(name='Non-Recurring Expenses', value='26824'), cls=<class 'panel.widgets.input.TextInput'>, fn=<function calculate_difference at 0x000001C913E85BC0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [28]:
last_month_expenses_chart = last_month_expenses.hvplot.bar(
    x='Category', 
    y='NTD_Amount', 
    height=250, 
    width=850, 
    title="Last Month Expenses",
    ylim=(0, 25000))

last_month_expenses_chart

:Bars   [Category]   (NTD_Amount)

In [29]:
df['Date'] = pd.to_datetime(df['Date'])           
df['Month-Year'] = df['Date'].dt.to_period('M')    
monthly_expenses_trend_by_cat = df.groupby(['Month-Year', 'Category'])['NTD_Amount'].sum().reset_index()

monthly_expenses_trend_by_cat['NTD_Amount']=monthly_expenses_trend_by_cat['NTD_Amount'].astype('str')
monthly_expenses_trend_by_cat['NTD_Amount']=monthly_expenses_trend_by_cat['NTD_Amount'].str.replace('-','')
monthly_expenses_trend_by_cat['NTD_Amount']=monthly_expenses_trend_by_cat['NTD_Amount'].astype('float')
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat["Category"].str.contains("unassigned") == False]

monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.sort_values(by='NTD_Amount', ascending=False)
monthly_expenses_trend_by_cat['NTD_Amount'] = monthly_expenses_trend_by_cat['NTD_Amount'].round().astype(int)
monthly_expenses_trend_by_cat['Month-Year'] = monthly_expenses_trend_by_cat['Month-Year'].astype(str)
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.rename(columns={'NTD_Amount': 'NTD_Amount '})

monthly_expenses_trend_by_cat

,Month-Year,Category,NTD_Amount
42,2023-09,Furnitures,36860
68,2023-12,Travels,20884
7,2023-02,Online Shopping,18272
62,2023-11,Sports,8542
10,2023-03,Education,7300
...,...,...,...
28,2023-07,Points Returned,200
64,2023-12,Digital Games,171
3,2023-01,Points Returned,130
74,2024-01,Points Returned,100


In [30]:
select_category1 = pn.widgets.Select(name='Select Category', options=[
    'All',
    'Sports',
    'Gas',
    'Groceries',
    'Restaurants',
    'Junk Food',
    'Delivery',
    'Online Shopping',
    'Clothes',
    'Furnitures',
    'Digital Games',
    'Miscellaneous',
    'Travels',
    'Car Rental',
    'Insurance',
    'Education',
    'Points Returned'
    #'Excluded'
])

select_category1

Select(options=['All', 'Sports', ...], value='All')

In [35]:
def plot_expenses(category):
    if category == 'All':
        plot_df = monthly_expenses_trend_by_cat.groupby('Month-Year').sum()
    else:
        plot_df = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat['Category'] == category].groupby('Month-Year').sum()
    plot = plot_df.hvplot.bar(x='Month-Year', y='NTD_Amount ')
    return plot

@pn.depends(select_category1.param.value)
def update_plot(category):
    plot = plot_expenses(category)
    return plot

monthly_expenses_trend_by_cat_chart = pn.Row(select_category1, update_plot)
monthly_expenses_trend_by_cat_chart[1].width = 600

monthly_expenses_trend_by_cat_chart

Row
    [0] Select(options=['All', 'Sports', ...], value='Gas')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False, width=600)

In [36]:
df = df[['Date', 'Category', 'Details', 'NTD_Amount']]
df['NTD_Amount']=df['NTD_Amount'].astype('str')
df['NTD_Amount']=df['NTD_Amount'].str.replace('-','')
df['NTD_Amount']=df['NTD_Amount'].astype('float')        #get absolute figures

df = df[df["Category"].str.contains("unassigned") == False]    #exclude "excluded" category
df['NTD_Amount'] = df['NTD_Amount'].round().astype(int)      #round values
df

,Date,Category,Details,NTD_Amount
0,2023-01-01,Junk Food,KFC-台中中清,165
2,2023-01-04,Digital Games,國外交易手續費,36
3,2023-01-04,Digital Games,GOOGLEANIPLEX,2373
4,2023-01-07,Junk Food,必勝客,1544
5,2023-01-07,Points Returned,點數折抵_必勝客,130
...,...,...,...,...
218,2024-02-10,Digital Games,Nintendo,60
219,2024-02-13,Junk Food,台灣肯德基KFCTa,548
220,2024-02-14,Junk Food,BURGERKING,417
221,2024-02-15,Delivery,優食-村早午餐,407


In [37]:
# Define a function to filter the dataframe based on the selected category
def filter_df(category):
    if category == 'All':
        return df
    return df[df['Category'] == category]

# Create a DataFrame widget that updates based on the category filter
summary_table = pn.widgets.DataFrame(filter_df('All'), height = 300,width=400)

# Define a callback that updates the dataframe widget when the category filter is changed
def update_summary_table(event):
    summary_table.value = filter_df(event.new)

# Add the callback function to the category widget
select_category1.param.watch(update_summary_table, 'value')

summary_table

DataFrame(height=300, sizing_mode='fixed', value=          Date  ..., width=400)

In [38]:
template = pn.template.FastListTemplate(
    title="Personal Credit Usage Summary",
    sidebar=[
        pn.pane.Markdown("## *Go With the Flow*"),
        pn.pane.PNG('587027.png', sizing_mode='scale_both'),
        pn.pane.Markdown(""),
        pn.pane.Markdown(""),
        select_category1
    ],
    main=[
        pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget, width=950),
        pn.Row(last_month_expenses_chart, height=240),
        pn.GridBox(
            monthly_expenses_trend_by_cat_chart[1],
            summary_table,
            ncols=2,
            width=500,  
            align='start',
            sizing_mode='stretch_width'
        )
    ]
)

template.show()

Launching server at http://localhost:7171
